In [1]:
import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import random as rn
import tensorflow as tf

rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')

# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False

# set Dropout
do = 0.3

if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss
# # remove outlier
# outlier = np.array([1674, 3427, 5086, 5093, 5096, 5104, 7821])
# mask = np.ones(len(vad), np.bool)
# mask[outlier] = 0
# vad = vad[mask]
# feat = feat[mask]
# API model, if use RNN, first two rnn layer must return_sequences=True


Using TensorFlow backend.


In [2]:
split = 7869

In [3]:
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(128, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    #net_speech = LSTM(64, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(dropout_rate)(net_speech)

    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_speech) for name in target_names]

    model = Model(input_speech, model_combined) 

    model.compile(loss=ccc_loss, 
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model

#def main(alpha, beta, gamma):
#model = api_model(0.1, 0.5, 0.4)
#model = api_model(1, 1, 1)
#model.summary()
# main function for for LOSO
# main function for for LOSO


In [4]:
# #speech_to_npy
# model1 = model()
# earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
#                               restore_best_weights=True)
# hist1 = model1.fit(feat[:split], vad[:split].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, 
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate(feat[split:], vad[split:].T.tolist())
# print(eval_metrik1)

# # make prediction
# predict = model1.predict(feat[6290:], batch_size=32)
# np.save('speech_npy7869', 
#          np.array(predict).reshape(3, 3749).T)


In [5]:
# main function for for LOSO
def main():
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:7869]],
                      vad[:7869].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[7869:]], vad[7869:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model_1.predict(feat[6290:], batch_size=32)
    np.save('npy7869/speech_7869-npy'+str(time)+'.npy',  
             np.array(predict).reshape(3, 3749).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main()
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('npy7869/speech_7869-1.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 4s 658us/step - loss: 1.8759 - v_loss: 0.9291 - a_loss: 0.4049 - d_loss: 0.5416 - v_ccc: 0.0707 - a_ccc: 0.5952 - d_ccc: 0.4583 - val_loss: 2.4933 - val_v_loss: 0.8448 - val_a_loss: 0.8494 - val_d_loss: 0.7924 - val_v_ccc: 0.1538 - val_a_ccc: 0.1495 - val_d_ccc: 0.2033
Epoch 2/50
6295/6295 [==============================] - 2s 358us/step - loss: 1.5840 - v_loss: 0.7547 - a_loss: 0.3614 - d_loss: 0.4682 - v_ccc: 0.2457 - a_ccc: 0.6385 - d_ccc: 0.5318 - val_loss: 2.0585 - val_v_loss: 0.8406 - val_a_loss: 0.6004 - val_d_loss: 0.6210 - val_v_ccc: 0.1592 - val_a_ccc: 0.4030 - val_d_ccc: 0.3793
Epoch 3/50
6295/6295 [==============================] - 2s 330us/step - loss: 1.5540 - v_loss: 0.7159 - a_loss: 0.3675 - d_loss: 0.4710 - v_ccc: 0.2848 - a_ccc: 0.6323 - d_ccc: 0.5290 - val_loss: 1.9723 - val_v_loss: 0

6295/6295 [==============================] - 2s 324us/step - loss: 1.0876 - v_loss: 0.3968 - a_loss: 0.2974 - d_loss: 0.3936 - v_ccc: 0.6032 - a_ccc: 0.7025 - d_ccc: 0.6067 - val_loss: 1.8616 - val_v_loss: 0.8001 - val_a_loss: 0.4712 - val_d_loss: 0.5905 - val_v_ccc: 0.2000 - val_a_ccc: 0.5281 - val_d_ccc: 0.4103
Epoch 25/50
2170/2170 [==============================] - 0s 123us/step
[0.1903305947780609, 0.5819482207298279, 0.438414990901947]
0
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 4s 638us/step - loss: 1.8860 - v_loss: 0.9403 - a_loss: 0.4108 - d_loss: 0.5357 - v_ccc: 0.0604 - a_ccc: 0.5892 - d_ccc: 0.4645 - val_loss: 2.3772 - val_v_loss: 0.8334 - val_a_loss: 0.7836 - val_d_loss: 0.7541 - val_v_ccc: 0.1660 - val_a_ccc: 0.2158 - val_d_ccc: 0.2410
Epoch 2/50
6295/6295 [==============================] - 2s 355us/step - loss: 1.6162 - v_loss: 0.7748 - a_loss: 0.3666 - d_loss: 0.4755 - v_ccc: 0.2252 - a_ccc: 0.6336 - d_ccc: 0

Epoch 24/50
6295/6295 [==============================] - 2s 322us/step - loss: 1.0973 - v_loss: 0.4068 - a_loss: 0.2949 - d_loss: 0.3955 - v_ccc: 0.5930 - a_ccc: 0.7053 - d_ccc: 0.6044 - val_loss: 1.8694 - val_v_loss: 0.7803 - val_a_loss: 0.4962 - val_d_loss: 0.5930 - val_v_ccc: 0.2182 - val_a_ccc: 0.5032 - val_d_ccc: 0.4092
Epoch 25/50
6295/6295 [==============================] - 2s 321us/step - loss: 1.0838 - v_loss: 0.3886 - a_loss: 0.2974 - d_loss: 0.3981 - v_ccc: 0.6113 - a_ccc: 0.7028 - d_ccc: 0.6022 - val_loss: 1.8447 - val_v_loss: 0.7824 - val_a_loss: 0.4700 - val_d_loss: 0.5927 - val_v_ccc: 0.2159 - val_a_ccc: 0.5300 - val_d_ccc: 0.4094
Epoch 26/50
6295/6295 [==============================] - 2s 329us/step - loss: 1.0729 - v_loss: 0.3878 - a_loss: 0.2950 - d_loss: 0.3898 - v_ccc: 0.6124 - a_ccc: 0.7049 - d_ccc: 0.6098 - val_loss: 1.8708 - val_v_loss: 0.8000 - val_a_loss: 0.4830 - val_d_loss: 0.5901 - val_v_ccc: 0.2002 - val_a_ccc: 0.5170 - val_d_ccc: 0.4120
Epoch 27/50
2170/21

Epoch 22/50
6295/6295 [==============================] - 2s 342us/step - loss: 1.1152 - v_loss: 0.4151 - a_loss: 0.2986 - d_loss: 0.4014 - v_ccc: 0.5848 - a_ccc: 0.7014 - d_ccc: 0.5987 - val_loss: 1.8203 - val_v_loss: 0.7870 - val_a_loss: 0.4556 - val_d_loss: 0.5843 - val_v_ccc: 0.2158 - val_a_ccc: 0.5446 - val_d_ccc: 0.4193
Epoch 23/50
6295/6295 [==============================] - 2s 354us/step - loss: 1.1225 - v_loss: 0.4154 - a_loss: 0.3038 - d_loss: 0.4044 - v_ccc: 0.5845 - a_ccc: 0.6969 - d_ccc: 0.5961 - val_loss: 1.8453 - val_v_loss: 0.7907 - val_a_loss: 0.4570 - val_d_loss: 0.6014 - val_v_ccc: 0.2115 - val_a_ccc: 0.5415 - val_d_ccc: 0.4017
Epoch 24/50
6295/6295 [==============================] - 2s 337us/step - loss: 1.0981 - v_loss: 0.4004 - a_loss: 0.2952 - d_loss: 0.4024 - v_ccc: 0.5996 - a_ccc: 0.7046 - d_ccc: 0.5977 - val_loss: 1.8194 - val_v_loss: 0.7968 - val_a_loss: 0.4463 - val_d_loss: 0.5783 - val_v_ccc: 0.2044 - val_a_ccc: 0.5523 - val_d_ccc: 0.4240
Epoch 25/50
6295/62

Epoch 13/50
6295/6295 [==============================] - 2s 321us/step - loss: 1.2110 - v_loss: 0.4658 - a_loss: 0.3122 - d_loss: 0.4333 - v_ccc: 0.5343 - a_ccc: 0.6882 - d_ccc: 0.5666 - val_loss: 1.8639 - val_v_loss: 0.8017 - val_a_loss: 0.4753 - val_d_loss: 0.5938 - val_v_ccc: 0.1986 - val_a_ccc: 0.5282 - val_d_ccc: 0.4093
Epoch 14/50
6295/6295 [==============================] - 2s 321us/step - loss: 1.2045 - v_loss: 0.4640 - a_loss: 0.3109 - d_loss: 0.4301 - v_ccc: 0.5362 - a_ccc: 0.6891 - d_ccc: 0.5702 - val_loss: 1.8742 - val_v_loss: 0.8434 - val_a_loss: 0.4642 - val_d_loss: 0.5711 - val_v_ccc: 0.1572 - val_a_ccc: 0.5372 - val_d_ccc: 0.4315
Epoch 15/50
6295/6295 [==============================] - 2s 319us/step - loss: 1.1794 - v_loss: 0.4505 - a_loss: 0.3092 - d_loss: 0.4189 - v_ccc: 0.5491 - a_ccc: 0.6907 - d_ccc: 0.5808 - val_loss: 1.8170 - val_v_loss: 0.7953 - val_a_loss: 0.4574 - val_d_loss: 0.5681 - val_v_ccc: 0.2032 - val_a_ccc: 0.5449 - val_d_ccc: 0.4350
Epoch 16/50
6295/62

...

Epoch 10/50
6295/6295 [==============================] - 2s 361us/step - loss: 1.2201 - v_loss: 0.4762 - a_loss: 0.3161 - d_loss: 0.4279 - v_ccc: 0.5239 - a_ccc: 0.6837 - d_ccc: 0.5722 - val_loss: 1.9026 - val_v_loss: 0.8054 - val_a_loss: 0.4970 - val_d_loss: 0.6054 - val_v_ccc: 0.1968 - val_a_ccc: 0.5044 - val_d_ccc: 0.3961
Epoch 11/50
6295/6295 [==============================] - 2s 363us/step - loss: 1.2163 - v_loss: 0.4760 - a_loss: 0.3150 - d_loss: 0.4257 - v_ccc: 0.5239 - a_ccc: 0.6851 - d_ccc: 0.5747 - val_loss: 1.8473 - val_v_loss: 0.7897 - val_a_loss: 0.4835 - val_d_loss: 0.5751 - val_v_ccc: 0.2109 - val_a_ccc: 0.5150 - val_d_ccc: 0.4268
Epoch 12/50
6295/6295 [==============================] - 2s 374us/step - loss: 1.2004 - v_loss: 0.4625 - a_loss: 0.3114 - d_loss: 0.4265 - v_ccc: 0.5376 - a_ccc: 0.6886 - d_ccc: 0.5734 - val_loss: 1.8151 - val_v_loss: 0.7994 - val_a_loss: 0.4507 - val_d_loss: 0.5626 - val_v_ccc: 0.2018 - val_a_ccc: 0.5456 - val_d_ccc: 0.4374
Epoch 13/50
6295/62

Epoch 18/50
6295/6295 [==============================] - 2s 374us/step - loss: 1.1368 - v_loss: 0.4309 - a_loss: 0.2975 - d_loss: 0.4082 - v_ccc: 0.5689 - a_ccc: 0.7024 - d_ccc: 0.5919 - val_loss: 1.8891 - val_v_loss: 0.8391 - val_a_loss: 0.4650 - val_d_loss: 0.5892 - val_v_ccc: 0.1607 - val_a_ccc: 0.5353 - val_d_ccc: 0.4150
Epoch 19/50
2170/2170 [==============================] - 0s 143us/step
[0.1942090094089508, 0.5936557650566101, 0.459333211183548]
15
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 8s 1ms/step - loss: 1.8404 - v_loss: 0.8918 - a_loss: 0.4155 - d_loss: 0.5333 - v_ccc: 0.1082 - a_ccc: 0.5847 - d_ccc: 0.4667 - val_loss: 2.4724 - val_v_loss: 0.8408 - val_a_loss: 0.8297 - val_d_loss: 0.7957 - val_v_ccc: 0.1578 - val_a_ccc: 0.1695 - val_d_ccc: 0.2004
Epoch 2/50
6295/6295 [==============================] - 2s 375us/step - loss: 1.5874 - v_loss: 0.7441 - a_loss: 0.3721 - d_loss: 0.4709 - v_ccc: 0.2559 - a_ccc: 0.6278

6295/6295 [==============================] - 2s 373us/step - loss: 1.4043 - v_loss: 0.5932 - a_loss: 0.3493 - d_loss: 0.4621 - v_ccc: 0.4066 - a_ccc: 0.6509 - d_ccc: 0.5381 - val_loss: 1.8854 - val_v_loss: 0.8260 - val_a_loss: 0.4705 - val_d_loss: 0.5905 - val_v_ccc: 0.1751 - val_a_ccc: 0.5295 - val_d_ccc: 0.4100
Epoch 6/50
6295/6295 [==============================] - 2s 372us/step - loss: 1.3387 - v_loss: 0.5552 - a_loss: 0.3344 - d_loss: 0.4493 - v_ccc: 0.4450 - a_ccc: 0.6658 - d_ccc: 0.5505 - val_loss: 1.8139 - val_v_loss: 0.7921 - val_a_loss: 0.4600 - val_d_loss: 0.5645 - val_v_ccc: 0.2091 - val_a_ccc: 0.5406 - val_d_ccc: 0.4364
Epoch 7/50
6295/6295 [==============================] - 2s 370us/step - loss: 1.3002 - v_loss: 0.5288 - a_loss: 0.3296 - d_loss: 0.4413 - v_ccc: 0.4711 - a_ccc: 0.6702 - d_ccc: 0.5585 - val_loss: 1.8116 - val_v_loss: 0.7840 - val_a_loss: 0.4549 - val_d_loss: 0.5750 - val_v_ccc: 0.2144 - val_a_ccc: 0.5465 - val_d_ccc: 0.4275
Epoch 8/50
6295/6295 [===========

Epoch 7/50
6295/6295 [==============================] - 3s 420us/step - loss: 1.3071 - v_loss: 0.5335 - a_loss: 0.3257 - d_loss: 0.4477 - v_ccc: 0.4667 - a_ccc: 0.6740 - d_ccc: 0.5522 - val_loss: 1.8432 - val_v_loss: 0.7855 - val_a_loss: 0.4814 - val_d_loss: 0.5832 - val_v_ccc: 0.2143 - val_a_ccc: 0.5225 - val_d_ccc: 0.4201
Epoch 8/50
6295/6295 [==============================] - 2s 394us/step - loss: 1.2929 - v_loss: 0.5238 - a_loss: 0.3228 - d_loss: 0.4456 - v_ccc: 0.4760 - a_ccc: 0.6771 - d_ccc: 0.5540 - val_loss: 1.8494 - val_v_loss: 0.7760 - val_a_loss: 0.4867 - val_d_loss: 0.5913 - val_v_ccc: 0.2223 - val_a_ccc: 0.5166 - val_d_ccc: 0.4117
Epoch 9/50
6295/6295 [==============================] - 3s 407us/step - loss: 1.2719 - v_loss: 0.5069 - a_loss: 0.3241 - d_loss: 0.4406 - v_ccc: 0.4929 - a_ccc: 0.6758 - d_ccc: 0.5594 - val_loss: 1.8588 - val_v_loss: 0.8101 - val_a_loss: 0.4751 - val_d_loss: 0.5799 - val_v_ccc: 0.1891 - val_a_ccc: 0.5289 - val_d_ccc: 0.4232
Epoch 10/50
6295/6295 

Epoch 4/50
6295/6295 [==============================] - 2s 395us/step - loss: 1.5190 - v_loss: 0.6896 - a_loss: 0.3616 - d_loss: 0.4676 - v_ccc: 0.3103 - a_ccc: 0.6385 - d_ccc: 0.5322 - val_loss: 1.9532 - val_v_loss: 0.8417 - val_a_loss: 0.5080 - val_d_loss: 0.6002 - val_v_ccc: 0.1567 - val_a_ccc: 0.4917 - val_d_ccc: 0.3984
Epoch 5/50
6295/6295 [==============================] - 2s 384us/step - loss: 1.4255 - v_loss: 0.6177 - a_loss: 0.3418 - d_loss: 0.4659 - v_ccc: 0.3822 - a_ccc: 0.6582 - d_ccc: 0.5341 - val_loss: 1.8908 - val_v_loss: 0.7904 - val_a_loss: 0.4980 - val_d_loss: 0.6062 - val_v_ccc: 0.2100 - val_a_ccc: 0.5037 - val_d_ccc: 0.3955
Epoch 6/50
6295/6295 [==============================] - 2s 388us/step - loss: 1.3481 - v_loss: 0.5625 - a_loss: 0.3327 - d_loss: 0.4528 - v_ccc: 0.4374 - a_ccc: 0.6673 - d_ccc: 0.5472 - val_loss: 1.7964 - val_v_loss: 0.7721 - val_a_loss: 0.4595 - val_d_loss: 0.5648 - val_v_ccc: 0.2291 - val_a_ccc: 0.5382 - val_d_ccc: 0.4362
Epoch 7/50
6295/6295 [